In [1]:
import wntr
import networkx as nx
import pickle
import matplotlib
import matplotlib.pyplot as plt
# "the default sans-serif font is Arial"
matplotlib.rcParams['font.sans-serif'] = "Arial"
# Then, "ALWAYS use sans-serif fonts"
# matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams.update({'font.size': 12})
import numpy as np
import copy
import wntr.network.controls as controls

In [2]:
# Get the non-zero expected demand
def get_demand_nodes(wdn):
    junctions = wdn.junction_name_list
    demand_nodes = []
    for j in junctions:
        j_object = wdn.get_node(j)
        base_demand = j_object.demand_timeseries_list[0].base_value
        if base_demand > 1e-8:
            demand_nodes.append(j)
    return demand_nodes

In [3]:
def pipefailure(wdn, uniform, mitigationlist, mitigationf):
    pipelist = wdn.pipe_name_list
    res = {}
    # First assign all the uniform ones
    for p in pipelist:
        res[p] = uniform
    # Update mitigated ones
    for p in mitigationlist:
        res[p] = mitigationf
    return res

In [4]:
def modifywdn(wdn, removelist):
    wdnc = copy.deepcopy(wdn)
    for pipe in removelist:
        wdnc.remove_link(pipe)
    return wdnc

In [5]:
def avg_function_loss(series_t, simulation_results, demand_nodes, demands):
    function_loss = 0
    # all the result demands
    actual_demand = simulation_results.node['demand']
    # Get the total expected demand at this time
    eds = {}
    r = 0
    for t in series_t:
        ed = 0
        ad = 0
        for dn in demand_nodes:
            ed += demands.loc[t*3600, dn]
            try:
                ad += actual_demand.loc[t*3600, dn]
            except:
                pass
        r += (1 - (ad / ed))
    return ( r/len(series_t) )

In [6]:
def physical(pipef, pipelist, wdn, conting_st, conting_et, series_t, demand_nodes, demands):
    # Generate failure states
    failed_pipes = []
    # Generate a list of random variables 
    random_list = np.random.uniform(low = 0.0, high = 1.0, size = len(pipelist))
    for j in range(len(pipelist)):
        # Determine the state
        if random_list[j] <= pipef[pipelist[j]]:
            failed_pipes.append(pipelist[j])
        # Only when there are failed pipes 
    if len(failed_pipes) == 0:
        r = 0
    else:
        # The corresponding simulation results after turning off a set of pipe
        # Make a copy of the original water distribution network
        wdnc = copy.deepcopy(wdn)
        ctrl1_list = []
        ctrl2_list = []
        for k in range(len(failed_pipes)):
            p_object = wdnc.get_link(failed_pipes[k])
            p_act1 = controls.ControlAction(p_object, 'status', 0)
            p_cond1 = controls.SimTimeCondition(wdnc, '=', str(conting_st) + ':00:00')
            ctrl1 = controls.Control(p_cond1, p_act1)
            ctrl1_list.append(ctrl1)
            p_act2 = controls.ControlAction(p_object, 'status', 1)
            p_cond2 = controls.SimTimeCondition(wdnc, '=', str(conting_et) + ':00:00')
            ctrl2 = controls.Control(p_cond2, p_act2)
            ctrl2_list.append(ctrl2)
        # Assign the controls on the network
        for m in range(len(ctrl1_list)):
            wdnc.add_control('Conting_start' + str(m), ctrl1_list[m])
            wdnc.add_control('Conting_end' + str(m), ctrl2_list[m])
        wdnc.options.time.duration = conting_et * 3600
        wdnc.options.hydraulic.demand_model = 'PDA'
        sim = wntr.sim.WNTRSimulator(wdnc)
        # Try the pressure driven simulation
        try:
            simulation_results = sim.run_sim()
            r = avg_function_loss(series_t, simulation_results, demand_nodes, demands)
        except:
            r = 1
    return r

### The scenario modeling with the state variable as inputs

In [7]:
def generate_scenario(x, xpipe, wdn):
    wdnc = copy.deepcopy(wdn)
    pipe_to_remove = []
    for i in range(len(x)):
        if x[i] == 0:
            pipe_to_remove.append(xpipe[i])
    for pipe in pipe_to_remove:
        wdnc.remove_link(pipe)
    return wdnc 

### Generate decision variables with the constraints  

In [8]:
def generate_decision(cmax, c, xprob):
    # Generare a uniform permutation of the decision variables
    a = []
    m = len(xprob) # the total number of decision variables
    for i in range(m):
        a.append(i)
    for i in range(m):
        index = np.random.randint(i, m, size = 1)[0]
        # Swap the element
        a[i], a[index] = a[index], a[i]
    # Generate the decision variable value
    total_c = 0 # the total cost so far
    x = np.zeros(m) # whether the corresponding pipe is selected 
    for k in range(m):    
        # generate the Bernouli variable 
        index = a[k]
        bernoulli = np.random.binomial(size = 1, n = 1, p = xprob[index])[0]
        # Update the total cost
        total_c += bernoulli * c[index] 
        # meet the budget constraints
        if total_c <= cmax:
            x[index] = bernoulli 
        else:
            x[index] = 0
            # all the left pipes are set to be 0
            break
    return x 

### The cross entropy optimization function

In [9]:
def cross_entropy(wdn, cmax, c, xprob, alllist, n1, alpha, beta, ro, uniform, 
                  augmentationf, conting_st, conting_et, series_t, demand_nodes, demands):
    # Initialize
    xprob_current = np.array(xprob)
    beta_hat = max(np.minimum(xprob_current, 1 - xprob_current))
    m = len(xprob)
    # Iterate until meeting the stopping criteria
    while beta_hat > beta:
        y = np.ones(n1)
        x = np.zeros((n1, m))
        for i in range(n1):
            # Generate random samples
            augmentationi = generate_decision(cmax, c, xprob_current)
            x[i] = augmentationi
            wdn_i = generate_scenario(augmentationi, alllist, wdn)
            # Perform hydraulic analysis
            augmentationlisti = [alllist[k] for k in range(len(augmentationi)) if augmentationi[k] == 1]
            pipef = pipefailure(wdn_i, uniform, augmentationlisti, augmentationf)
            pipelist = wdn_i.pipe_name_list
            y[i] = physical(pipef, pipelist, wdn_i, conting_st, conting_et, series_t, demand_nodes, demands)
        # Update the gamma
        gamma_current = np.quantile(y, ro)
        # Update the xprob
        numerator = np.zeros(m)
        denomenator = 0
        for i in range(n1):
            if y[i] <= gamma_current:
                numerator += x[i]
                denomenator += 1
        xprob_hat = numerator / denomenator
        # Smooth parameter
        xprob_current = alpha * xprob_hat + (1 - alpha) * xprob_current
        print(xprob_current)
        beta_hat = max(np.minimum(xprob_current, 1 - xprob_current))
    return xprob_current

### Load the essential data

In [10]:
Wdn_name = 'Net3S'
Wdn = wntr.network.WaterNetworkModel('Net3_ag_1004.inp')
Wdnc = copy.deepcopy(Wdn)
Wdnc.options.time.duration = 24 * 3600
Wdnc.options.hydraulic.demand_model = 'PDA'
Simc = wntr.sim.WNTRSimulator(Wdnc)
Results = Simc.run_sim() # by default run EPANET 2.2
Demands = Results.node['demand']
Conting_st = 0
Conting_et = 24
Series_t = [int(t) for t in np.linspace(1, 23, 23)]
Nodes = get_demand_nodes(Wdnc)

### The basic parameters

In [11]:
# The budget, divided by 100
Cmax = 4000
# Costs, probs
Alllist = ['add1', 'add26', 'add27', 'add9', 'add20', 'add24', 'add7', 'add13', 'add15', 'add21']
Costlist = np.array([1106.7694850441985,
 243.74809891566375,
 973.056758994562,
 565.3659599100389,
 759.3663740250817,
 440.7971661093114,
 793.2837131978698,
 386.1476267245468,
 468.4390183631158,
 136.82174909348328])
Uniform = 0.054
Augmentationf = 0.002
Xprob = 0.5 * np.ones(len(Alllist))
# The ro parametr for cross entropy update
Ro = 0.1
# The sample size for CE
N1 = 1000
# The smooth parameter
Alpha = 0.7
# The stop parameter 
Beta = 0.05

In [ ]:
X = cross_entropy(Wdn, Cmax, Costlist, Xprob, Alllist, N1, Alpha, Beta, Ro, Uniform, Augmentationf, Conting_st, Conting_et, Series_t, Nodes, Demands)

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 10:54:29. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 23:00:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 01:17:50. Line search failed at iteration 67
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:25:39. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\an

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:01. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:37:28. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 01:25:00. Line search failed at iteration 25
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:37:32. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded max

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 11:00:00. Line search failed at iteration 53
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:43:58. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 02:20:49. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 05:00:31. Line search failed at iteration 44
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:13

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:42:35. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 12:00:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 12:04:39. Line search failed at iteration 40
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:43:59. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\co

[0.542 0.507 0.584 0.493 0.465 0.619 0.626 0.437 0.444 0.5  ]


...

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:43:57. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:02. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:24. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:25. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:25. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:30. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:25. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 11:00:00. Line search failed at iteration 44
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded max

[9.896e-01 5.938e-01 9.991e-01 1.804e-02 5.201e-04 9.992e-01 9.995e-01 1.378e-01 2.476e-05 2.753e-01]


C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:21. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:22. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:43:32. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 03:58:04. Line search failed at iteration 45
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation d

C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:25. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1299: UserWarning: Simulation did not converge at time 06:15:00. Reached maximum number of iterations: 2999
  warnings.warn('Simulation did not converge at time ' + self._get_time() + '. ' + mesg)
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:44:07. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exceeded maximum number of trials at time 00:40:31. 
  warnings.warn('Exceeded maximum number of trials at time ' + self._get_time() + '. ')
C:\Users\yuanc\anaconda3\lib\site-packages\wntr\sim\core.py:1326: UserWarning: Exce

In [ ]:
FRes = open('Net3_Augmentation4000_CE_1023.pickle','wb')
pickle.dump(X, FRes)
FRes.close()